# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (сопоставимые или большие по объему). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг <start>  
    - еще одна матрица не нужна, можно по строкам хронить биграмы, а по колонкам униграммы  
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так).

In [79]:
from string import punctuation
from collections import Counter
from razdel import tokenize as razdel_tokenize
from nltk.tokenize import sent_tokenize
from scipy.sparse import lil_matrix
import numpy as np

In [80]:
news = open('lenta.txt', encoding='UTF8').read()

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

sentences_news = [['<start>', '<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news[:5000000])]

unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))
    trigrams_news.update(ngrammer(sentence, n=3))

bigrams_news['<start> <start>'] = len(sentences_news)

matrix_news = lil_matrix((len(bigrams_news), len(unigrams_news)))

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}

id2bigram_news = list(bigrams_news)
bigram2id_news = {bigram:i for i, bigram in enumerate(id2bigram_news)}

for ngram in trigrams_news:
    word1, word2, word3 = ngram.split()
    bigram = word1 + ' ' + word2
    matrix_news[bigram2id_news[bigram], word2id_news[word3]] =  (trigrams_news[ngram] / bigrams_news[bigram])

def generate(matrix, id2word, bigram2id, n=100, start='<start> <start>'):
    text = []
    current_bigram = start

    for i in range(n):
        chosen = np.random.choice(matrix.shape[1], p=matrix[bigram2id[current_bigram]].toarray()[0])
        text.append(id2word[chosen])

        if id2word[chosen] == '<end>':
            current_bigram = '<start> <start>'
        else:
            current_bigram = current_bigram.split()[1] + ' ' + id2word[chosen]

    return ' '.join(text)

In [85]:
for i in range(5):
    print('Text', i+1, ':')
    print(generate(matrix_news, id2word_news, bigram2id_news).replace('<end>', '\n'))
    print()

Text 1 :
основная причина распространения болезни отсутствие достаточных средств продолжать кампанию было бы странно если бы эти взрывы произошли то дома превратились бы в том числе 10 детей 
 оформлением храма занимались творческие коллективы под руководством подполковника милиции абдулманапа мусаева 
 структура договоренности уже согласована почти полностью контролируют старопромысловский район и организовали эвакуацию жителей соседних домов были срочно эвакуированы 
 между тем соперник мэра иван стариков имеет достаточно теплые отношения прокуратура санкт-петербурга уже заявила журналистам риа новости игорь шабдурасулов о котором было заявлено на совместной пресс-конференции в воронеже охарактеризовал общую криминальную ситуацию в республике 
 по окончании встречи прошедшей в пятницу прекращена

Text 2 :
россия каждый раз выражает свое принципиальное согласие баллотироваться на пост мэра столицы оставлено в силе вердикт суда первой инстанции признал незаконным и не может гарантирова

In [90]:
def perplexity(logp, N):
    return np.exp((-1/N) * logp)


def compute_joint_proba_trigram(text, matrix, word2id, bigram2id):
    c_prob = 0
    tokens = ['<start>', '<start>'] + normalize(text) + ['<end>']
    for i in range(len(tokens)-2):
        c_word1 = tokens[i]
        c_word2 = tokens[i+1]
        c_word3 = tokens[i+2]
        bigram = c_word1 + ' ' + c_word2
        if bigram in bigram2id and c_word3 in word2id:
            p = matrix[bigram2id[bigram], word2id[c_word3]]
            if p != 0:
                c_prob += np.log(p)
            else:
                c_prob += np.log(1e-6)
        else:
            c_prob += np.log(1e-6)
    return c_prob, len(tokens)

ps = []
for sent in sent_tokenize(news[5000000:5000050]):
    t_prob, N = compute_joint_proba_trigram(sent, matrix_news, word2id_news, bigram2id_news)
    if not N:
        continue
    ps.append(perplexity(t_prob, N))
print('Perplexity:', np.mean(ps))

Perplexity: 11388.14815381501


## Задание № 2* (2 балла). 

Прочитайте главу про языковое моделирование в книге Журафски и Мартина - https://web.stanford.edu/~jurafsky/slp3/3.pdf

Развернуто (в пределах 1000 знаков) ответьте на вопросы (по-русски):

1. Что можно делать с проблемой несловарных слов? В семинаре мы просто использовали какое-то маленькое значение вероятности, а какие есть другие способы?

Проблема несловарных слов возникает, когда в тестовых данных встречаются слова, которых нет в обучающих данных. Один из способов решения этой проблемы - использование закрытой системы словарей, где тестовые данные могут содержать только слова из
известного словаря. Однако в большинстве реальных ситуаций мы имеем дело с несловарными словами. Один из способов создания открытой системы словарей - моделирование потенциальных неизвестных слов в тестовых данных путем добавления псевдо-слова
<UNK>.

Существует два распространенных способа обучения вероятностей неизвестного слова <UNK>. Первый - это превращение проблемы обратно в закрытую систему словарей путем выбора фиксированного словаря заранее. Второй альтернативой, когда у нас нет заранее
определенного словаря, является создание такого словаря неявно, заменяя слова в обучающих данных на <UNK> на основе их частоты.

Точный выбор <UNK> влияет на метрики, такие как перплексия. Языковая модель может достичь низкой перплексии, выбрав маленький словарь и назначив неизвестному слову высокую вероятность. Таким образом, перплексии могут быть сравнены только между языковыми
моделями с одинаковыми словарями.

2. Что такое сглаживание (smoothing)?

 Чтобы языковая модель не присваивала нулевую вероятность невиданным событиям(слова, которые есть в словаре, но встречаются в тестовых данных в невиданном контексте. Например, они появляются после слова, после которого они никогда не появлялись в
 обучающих данных), мы должны отнять немного вероятностной массы у более частых событий и отдать ее событиям, которые мы никогда не видели. Эта модификация называется сглаживанием или
 уценкой.
 Есть различные способы сглаживания: сглаживание Лапласа (add-one), сглаживание add-k, глупый откат (stupid backoff) и сглаживание Кнезера-Нея.